In [24]:
import numpy as np
import random
from scipy.optimize import minimize

In [25]:
# 単位ベクトルの生成
def generate_unit_vectors():
    k = np.array(random.sample(range(1, 100), 3))
    n = k / np.linalg.norm(k)
    return n, generate_perpendicular_unit_vector(n)

# 磁場ベクトルの生成
def generate_perpendicular_unit_vector(n):
    if np.all(n == 0):
        raise ValueError("The input vector n should not be the zero vector.")
    
    if abs(n[0]) < abs(n[1]) and abs(n[0]) < abs(n[2]):
        w = np.array([1, 0, 0])
    elif abs(n[1]) < abs(n[0]) and abs(n[1]) < abs(n[2]):
        w = np.array([0, 1, 0])
    else:
        w = np.array([0, 0, 1])
    
    perp_vec = np.cross(n, w)
    return perp_vec / np.linalg.norm(perp_vec)

n, s = generate_unit_vectors()
c = np.cross(n, s)

t = c*s

print(t)

[-0.05212017 -0.          0.05212017]


In [26]:
import sympy as sp
from scipy.optimize import minimize
import numpy as np

# シンボリック変数の定義
K_xx, K_xy, K_xz, K_yy, K_yz, K_zz, theta, s_x, s_y, s_z = sp.symbols('K_xx K_xy K_xz K_yy K_yz K_zz theta s_x s_y s_z')

# 回転行列の定義
R = sp.Matrix([
    [sp.cos(theta), sp.sin(theta), 0],
    [sp.sin(theta), sp.cos(theta), 0],
    [0, 0, 1]
])

# ベクトルの定義
s = sp.Matrix([s_x, s_y, s_z])

# 行列Kの定義
K = sp.Matrix([
    [K_xx, K_xy, K_xz],
    [K_xy, K_yy, K_yz],
    [K_xz, K_yz, K_zz]
])

# K_{z'z'}の定義 (シンボリック)
K_z_prime_z_prime = (R * s).T * K * (R * s)

# K_{z'z'}の右辺の定義
K_zz_sym = (K_xx + K_yy)/2 + ((K_xx - K_yy)/2)*sp.cos(2*theta) + K_xy*sp.sin(2*theta)

# 単位ベクトルの制約条件
constraint_unit_s = sp.Eq(s_x**2 + s_y**2 + s_z**2, 1)

# 定数の定義 (実際の値を入力してください)
K_xx_val = 1.0
K_xy_val = 0.5
K_xz_val = 0.3
K_yy_val = 1.2
K_yz_val = 0.4
K_zz_val = 1.5
theta_val = np.pi / 6  # 例えば30度の回転

# 目的関数の定義
def objective_function(params):
    s_x_val, s_y_val, s_z_val = params
    K_z_prime_z_prime_val = K_z_prime_z_prime.subs({
        K_xx: K_xx_val,
        K_xy: K_xy_val,
        K_xz: K_xz_val,
        K_yy: K_yy_val,
        K_yz: K_yz_val,
        K_zz: K_zz_val,
        theta: theta_val,
        s_x: s_x_val,
        s_y: s_y_val,
        s_z: s_z_val
    })
    K_zz_val_calculated = K_zz_sym.subs({
        K_xx: K_xx_val,
        K_xy: K_xy_val,
        K_xz: K_xz_val,
        K_yy: K_yy_val,
        K_yz: K_yz_val,
        K_zz: K_zz_val,
        theta: theta_val
    })
    return (K_z_prime_z_prime_val[0] - K_zz_val_calculated)**2

# 制約条件の定義
def constraint_unit_s(params):
    s_x_val, s_y_val, s_z_val = params
    return s_x_val**2 + s_y_val**2 + s_z_val**2 - 1

# 初期パラメータの推定値
params_init = np.random.rand(3)

# 制約条件をまとめる
constraints = [{'type': 'eq', 'fun': constraint_unit_s}]

# 最小二乗法によるパラメータ推定
result = minimize(objective_function, params_init, constraints=constraints)

# 推定されたパラメータ
s_x_val, s_y_val, s_z_val = result.x
print(f"s_x = {s_x_val:.4f}")
print(f"s_y = {s_y_val:.4f}")
print(f"s_z = {s_z_val:.4f}")


s_x = 0.5355
s_y = -0.1774
s_z = 0.8257
